# Hands-on Introduction to Python And Machine Learning

Instructor: Tak-Kei Lam

(Readers are assumed to have a little bit programming background.)


### 2. Similarity-based algorithms
When dealing with data, we often need to define how much a data point is similar to other data points. If we know how similar or dissimilar any two data points are, we can then divide the data points into groups where the elements in the same group are similar. and thus achieve machine learning. *Classification* and *clustering* are two groups of similarity-based machine learning algorithms. How good the classification or clustering result is almost always depends on how suitable the similarity metric is.

Classification algorithms are supervised learning machine learning algorithms, whereas clustering algorithms are unsupervised. 

Example of classification algorithms:
- k-nearest neighbours

Examples of clustering algorithms:
- k-means
- Expectation-maximization

#### K-nearest neighbours
K-nearest neighbours is a supervised classification algorithm. Its procedures:
1. Build a database of samples whose classes are known (the samples have been labelled)
2. If we want to classify an unseen piece of data $d$, we need to:
    1. Find the $k$ most similar samples in the database, which are known as the $k$-nearest neighbours, and then
    2. Find the majority class of the $k$ nearest neighbours, and assign it to $d$

Let write our simplest implementation of k-nearest neighbours in Python from scratch!

In [ ]:
# Importing libraries
import numpy as np

def euclideanDistance(data1, data2):    
    """ calculate sum of the squares of each column """
    total_distance = np.sum(np.square(data1[1] - data2[1]))
    eucliden = np.sqrt(total_distance)
    return eucliden


def kNearestNeighboursPredict(trainingData, k, testData):
    """ Inputs:
            trainingData is an array of tuples
            k specifies the number of nearest neighbours
            testData is an array of tuples

            a trainingData should be a tuple (class, numpy array of features)
            so trainingData is [(class 1, numpy array of features),
                                (class 2, numpy array of features),
                                ...]

            the format of testData is the same as trainingData

        Calculates the class of the given data using Eucliden distance as the similarity metric.
    """
    predicted_classes = []
    
    for d in testData:
        # calculate the Euclidean distance
        distances = [(tr, euclideanDistance(d, tr)) for tr in trainingData]
        
        # 
        sorted_distances = sorted(distances, key=lambda tup: tup[1])[:k]

        kNeighbours = [d[0] for d in sorted_distances]        

        classes = {}
        for n in kNeighbours:
            if n[0] not in classes:
                classes[n[0]] = 1
            else:
                classes[n[0]] += 1                
        
        classes = sorted(classes.items(), key=lambda entry: entry[1])        
        
        predicted_classes.append(classes[0][0])
        
    return predicted_classes
                
trainingData = [('A', np.array([1])), ('A', np.array([2])),
                ('B', np.array([11])), ('B', np.array([12]))]

testData = [('X', np.array([13])), ('X', np.array([4]))]

predictedData = kNearestNeighboursPredict(trainingData, 2, testData)

print(predictedData)

Using scikit-learn is more convenient...

In [ ]:
# given 'HP', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def', 'Speed', can we figure out the Type of the Pokemon?

import matplotlib.pyplot as plt
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import mean_squared_error, r2_score
import pandas as pd

# Load the dataset
pokemons = pd.read_csv("pokemon.csv")
print(pokemons['Type 1'].unique())

pokemons = pokemons.sample(frac=1) # .sample(frac=1) randomize the data, and select 100% from the randomized

label_column = ['Type 1']
features_columns = ['HP', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def', 'Speed']

pokemons_features = pokemons[features_columns]
pokemons_label = pokemons[label_column]

# normalise every columns in pokemons_features
# pokemons_features = pokemons_features.apply(lambda x: (x - x.min())/(x.max() - x.min()))

# .values convert the datastructure from pandas's dataframe into numpy's array

# Split the data into training/test sets
last_index = -int(0.20*len(pokemons_features))
pokemons_features_train = pokemons_features[:last_index].values
pokemons_features_test = pokemons_features[last_index:].values 

last_index = -int(0.20*len(pokemons_label))
pokemons_label_train = pokemons_label[:last_index].values.flatten() # the expected labels
pokemons_label_test = pokemons_label[last_index:].values.flatten()  # the expected labels

# Create a k-nearest neighbours classifier
neigh = KNeighborsClassifier(n_neighbors=20)

# Train the model using the training sets
neigh.fit(pokemons_features_train, pokemons_label_train) 

# Make predictions using the testing set
pokemons_label_pred = neigh.predict(pokemons_features_test) # the actual labels

correct = 0.0
for i in range(0, len(pokemons_label_test)):
    print('expected {} VS. actual {}'.format(pokemons_label_test[i], pokemons_label_pred[i]))
    if pokemons_label_pred[i] == pokemons_label_test[i]:
        correct = correct+1
print('Accuracy: {}%'.format(correct/len(pokemons_label_test) * 100))

# What if we change k, and/or select a different set of features, and/or limit the number of Type?

You may have already noticed that the choice of the parameters such as $k$ and the features greatly affect the performance of k-nearest neighbours. Indeed, the parameters of the model are as important as the (dis)similarity metric in machine learning algorithms. Selecting the most suitable parameters is a big research topic per se.

#### K-means
K-means is an unsupervised clustering algorithm. It is different from k-nearest neighours, but k-nearest neighours is sometimes applied to implement k-means (as shown below). The procedures of k-means are listed below:
1. Guess $k$ cluster centres
2. For each data point, assign it to one of the closest clusters. Here a similarity metric defining the distance betweeen a data point and a cluster centre is required.
3. Update the centres of the clusters
4. Repeat step 2-3 until the centres of the clusters do not change

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# ---------------------------- the same functions defined for k-nearest neighbours BEGIN

def euclideanDistance(data1, data2):    
    """ calculate sum of the squares of each column """
    total_distance = np.sum(np.square(data1[1] - data2[1]))
    eucliden = np.sqrt(total_distance)
    return eucliden


def kNearestNeighboursPredict(centres, k, testData):
    """ Inputs:
            centres is an array of tuples
            k specifies the number of nearest neighbours
            testData is an array of tuples

            a centres should be a tuple (class, numpy array of features)
            so centres is [(class 1, numpy array of features),
                                (class 2, numpy array of features),
                                ...]

            the format of testData is the same as centres

        Calculates the class of the given data using Eucliden distance as the similarity metric.
    """
    predicted_classes = []
    
    for d in testData:
        # calculate the Euclidean distance
        distances = [(tr, euclideanDistance(d, tr)) for tr in centres]
        
        # 
        sorted_distances = sorted(distances, key=lambda tup: tup[1])[:k]

        kNeighbours = [d[0] for d in sorted_distances]        

        classes = {}
        for n in kNeighbours:
            if n[0] not in classes:
                classes[n[0]] = 1
            else:
                classes[n[0]] += 1                
        
        classes = sorted(classes.items(), key=lambda entry: entry[1])        
        
        predicted_classes.append(classes[0][0])
        
    return predicted_classes
# ---------------------------- the same functions defined for k-nearest neighbours END

def kmeansFit(data, k):
    # generate k random centres
    rand_int = np.random.randint(-100, 100, 1)[0]
    centre_values = [-rand_int, rand_int]
     # food for thought, why do we pick the initial centre values in this way?
    # how about  randomly generating them, say, from the range [-100, 100] instead?
    print('initial random centre values: {}'.format(centre_values))

     # centres is an array  in the form [ (classA, [numpy array of features]), (classB, [numpy array of features]), ...]
    centres = []
    for i, c in enumerate(centre_values):
        centres.append((i, np.array([c])))

    prev_centres = None

    classes = {}

    while True:
        # assign a class to every data
        assigned_classes = kNearestNeighboursPredict(centres, 1, data)
        print(assigned_classes)

        # store the class info as a dictionary in the following format:
        # { class A: array of data, class B: array of data, ...}
        for i in range(0, k):
            classes[i] = []                              
        for i, c in enumerate(assigned_classes):
            data[i] = (c, data[i][1])
            classes[assigned_classes[i]].append(data[i])

        # update the centres of every cluster
        for c, elements in classes.items():
            sum = np.zeros((len(data[0][1])))
            for e in elements:
                    sum += e[1]
            if (len(elements) > 0):
                mean = sum / len(elements)
                centres[c] = (c, mean)

        for c in centres:
            print('centre: {} has mean {}'.format(c[0], c[1]))

         # check if the centres are updated
        hasGreatlyChanged = False
        if prev_centres:
            for i, c in enumerate(centres):
                    diff = np.sum(np.absolute(prev_centres[i][1]-centres[i][1]))
                    print('prev: {} cur : {}', prev_centres[i][1], centres[i][1])
                    if diff > 0.5:
                        hasGreatlyChanged = True
                        break
        else:
            hasGreatlyChanged = True

        if not hasGreatlyChanged:
            break

        prev_centres = centres[0:]
        # why do we have to do this???
        # can we simply do: prev_centres = centres ??? 
        
        yield classes # we haven't learnt using yield yet. Can you guess what it is used for?

    #return classes 

# let's test our implementation
                               
# random data set 1 that consists of 10 integers in the range [-2, 4]
data_1 = np.random.randint(-1, 5, 10)
# random data set 2 that consists of 10 integers in the range [9, 15]
data_2 = np.random.randint(9, 16, 10)
# shuffle the concatenation of data_1 and data2
data_array = np.concatenate((data_1, data_2))
np.random.shuffle(data_array)

# just assign a dummy class for each data (for format compatibility)
data_array = [(0, np.array([d])) for i,d in enumerate(data_array)]

iterations=0
for predictedClasses in kmeansFit(data_array, 2):
    print('------------------------ Iteration: {}'.format(iterations))
    #print(predictedClasses)


    plt.figure(figsize=(10,8), dpi=100)
    for c,dataPoints in predictedClasses.items():
        if c == 0:
            colour = 'red'
        elif c == 1:
            colour = 'blue'

        x = [x[1] for x in dataPoints]
        y = np.zeros(len(x))
        plt.title('Clustering result at iteration: '+ str(iterations))
        plt.scatter(x, y, color=colour)
    iterations = iterations +1
    
plt.show()

k-means using Euclidean distance as the similarity metric is defined in mathematics notations as follows:

Given $n$ data points: $\{d_1, d_2, d_3, ..., d_n\}$, assign every data point $d_i$ to a cluster $C_i$ out of $k$ clusters such that $\sum_1^k{\sum_{d_i \in C_j}{||d_i - \mu _j||_2}}$ is minimum, where $\mu _j$ is the mean of all data points in the cluster $C_j$.

In this example, some important and valuable knowledge from human are used to train the model:
1. An educated guess of the initial centre values
2. An educated guess of the number of clusters
3. An educated guess of the stopping conditions

We can see from the examples of k-nearest neighbours (supvervised classification) and k-means (unsupervised clustering) that human knowledge is required in some sense, either explicit (through data labelling) or implicit (guesses of the parameters), to train the machine learning algorithms no matter they are supervised or unsupervised. Therefore the domain knowledge of a problem is actually very important.

That implies some people probably still have jobs even if artificial intelligence has dominated the universe! (seems legit...)

Using k-means in scikit-learn:

In [ ]:
# given 'HP', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def', 'Speed', can we figure out the Type of the Pokemon?

import matplotlib.pyplot as plt
import numpy as np
from sklearn.cluster import KMeans
from sklearn.metrics import mean_squared_error, r2_score
import pandas as pd    

# Load the dataset
pokemons = pd.read_csv("pokemon.csv")
print(pokemons['Type 1'].unique())

pokemons = pokemons.sample(frac=1) # .sample(frac=1) randomize the data, and select 100% from the randomized

label_column = ['Type 1']
features_columns = ['HP', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def', 'Speed']

pokemons_features = pokemons[features_columns]
pokemons_label = pokemons[label_column]

# normalise every columns in pokemons_features
# pokemons_features = pokemons_features.apply(lambda x: (x - x.min())/(x.max() - x.min()))

# .values convert the datastructure from pandas's dataframe into numpy's array

# Split the data into training/test sets
last_index = -int(0.20*len(pokemons_features))
pokemons_features_train = pokemons_features[:last_index].values
pokemons_features_test = pokemons_features[last_index:].values 

last_index = -int(0.20*len(pokemons_label))
pokemons_label_train = pokemons_label[:last_index].values.flatten() # the expected labels
pokemons_label_test = pokemons_label[last_index:].values.flatten()  # the expected labels

kmeans = KMeans(n_clusters=18, random_state=0).fit(pokemons_features_train)

# Make predictions using the testing set
pokemons_label_pred = kmeans.predict(pokemons_features_test) # the actual labels

correct = 0.0
for i in range(0, len(pokemons_label_pred)):
    print('Pokemon: {} actual type: {}'.format(pokemons.loc[i, 'Name'], pokemons_label_pred[i]))

#### Expectation-Maximization

In k-means (clustering), we use a technique in which the centres are guessed, the data points are assigned the guessed centres, then the centres are updated according to the information that has been gathered thus far, and the procedure repeats until the result *converges*. This algorithm is actually a way of making educated and systematic trial-and-error guesses. There is a highly similar algorithm known as Expectation-Maximization.

Expectation-Maximization is usually said to the the "soft" version of k-means. That means, instead assigning a data point to one and only one cluster, *a data point is assigned to all clusters with probability densities*; instead of characterising a cluster by its mean, a cluster is to be characterised by a *probability density function* or *mixtures of probability density functions*. The suitability that a data point belongs to a cluster depends on how it fits with the probability density functions of the cluster.

We usually assume the probability density function is of [Gaussian](https://en.wikipedia.org/wiki/Normal_distribution).

The two main steps of expectation-maximization:
1. Expectation, the E step: for each data point $d_i$, compute the probability density of it belonging to each of the clusters give the current model $m$
2. Maximization, the M step: update the model $m$ such that every data point can be assigned to a cluster with the greatest probability density (in other words, to make clusters further away from each other)

A program is worth a thousand words. Here we go:

In [ ]:
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt

def estimate_mean(data, weight):
    return np.sum(data * weight) / np.sum(weight)

def estimate_std(data, weight, mean):
    variance = np.sum(weight * (data - mean)**2) / np.sum(weight)
    return np.sqrt(variance)

# --------------------------------------------------- initialization
np.random.seed(110) # for reproducible random results

# set parameters
red_mean = 3
red_std = 0.8

blue_mean = 7
blue_std = 2

# draw 20 samples from normal distributions with red/blue parameters
red = np.random.normal(red_mean, red_std, size=20)
blue = np.random.normal(blue_mean, blue_std, size=20)

both_colours = np.sort(np.concatenate((red, blue))) # combine the points together

# From now on, assume we don't know how the data points were generate. We also don't know which cluster a data belongs to.


# ------------ initial guess 
# estimates for the mean
red_mean_guess = 1.1
blue_mean_guess = 9

# estimates for the standard deviation
red_std_guess = 2
blue_std_guess = 1.7

# ------------ expectation & maximization
# graph
g=[i for i in range(-10, 20)]

for i in range(0, 10):
    # just for plotting
    plt.figure(figsize=(10, 6), dpi=100)    
    plt.title("Red VS. Blue (Gaussian distribution) Take: " + str(i))
    red_norm = stats.norm(red_mean_guess, red_std_guess).pdf(g)
    blue_norm = stats.norm(blue_mean_guess, blue_std_guess).pdf(g)
    plt.scatter(g, red_norm)
    plt.scatter(g, blue_norm)
    plt.plot(g, red_norm, c='red')
    plt.plot(g, blue_norm, c='blue')
    

    # how likely is a data point belong to the clusters?
    likelihood_of_red = stats.norm(red_mean_guess, red_std_guess).pdf(both_colours)
    likelihood_of_blue = stats.norm(blue_mean_guess, blue_std_guess).pdf(both_colours)

    # new estimates of standard deviation
    likelihood_total = likelihood_of_red + likelihood_of_blue
    red_weight = likelihood_of_red / likelihood_total
    blue_weight = likelihood_of_blue / likelihood_total

    red_std_guess = estimate_std(both_colours, red_weight, red_mean_guess)
    blue_std_guess = estimate_std(both_colours, blue_weight, blue_mean_guess)

    # new estimates of mean
    red_mean_guess = estimate_mean(both_colours, red_weight)
    blue_mean_guess = estimate_mean(both_colours, blue_weight)

    plt.show()

##### Do you agree?

Both k-means and expectation-maximization can acutally be used for semi-supervised machine learning in which only a portion of the data is labelled.